# Population and CO2

### Building a visualization similar to this app http://worldpopulationhistory.org/map/1/mercator/1/0/25/

Using a datafile scraped from their site

### Imports and function definitions

In [394]:
#This directory contains all the data needed for the module. It should be in the same directory as the notebook
dataFolder = "data" 
import sys, os, shutil, errno, string, urllib


sys.path.append(( os.path.abspath( os.path.join(os.path.realpath("__file__"),  os.pardir, os.pardir) )))
import uvmodlib.v1 as uvmod

In [395]:
# Run this if you want to build in 3.0 rather than 2.0
from config import Settings
Settings.uvcustommodulelocation='C:\\Users\\msubbarao\\SCISS\\Uniview Theater 3.0\\Custom Modules'

### USES Conf Template for Population Animation

In [396]:
popTemplate='''mesh
{
    data popData    ./modules/$folderName/pop.uvstream
    data co2Data    ./modules/$folderName/co2.uvstream

    cullRadius $cr
    glslVersion 330

    propertyCollection
    {
        __objectName__
        {
            
            bool simBindRealtime False | public | desc "bind the time of this model to the actual time in Uniview (0=no, 1=yes)"
            vec1f showTime 2018 | public | desc "show the population at this date" | widget slider | range 1800 2100
            vec1f popSize 4.0 | public | desc "population marker size" | widget slider | range 0 6
            vec4f popColor 1 1 0 0.5 | public | desc "population marker color" | widget color_rgba 
            vec1f co2Size 2.5 | public | desc "co2 marker size" | widget slider | range 0 5
            vec4f co2Color 1 0 0 1.0 | public | desc "co2 marker color" | widget color_rgba 

        }

    }

    
    pass #population
    {
        passEnable true

        useDataObject popData

        shader
        {
            type defaultMeshShader
            {        
                stateManagerVar __objectName__.Alpha                uv_alpha
                StateManagerVar __objectName__.simBindRealtime simBindRealtime
                StateManagerVar __objectName__.showTime showTime
                StateManagerVar __objectName__.popSize popSize
                StateManagerVar __objectName__.popColor popColor

                vertexShader ./modules/$folderName/pop_shader.vs
                fragmentShader ./modules/$folderName/pop_shader.fs                
                geometryShader ./modules/$folderName/pop_shader.gs

                glState
                {
                    UV_CULL_FACE_ENABLE true
                    UV_DEPTH_ENABLE false
                    UV_BLEND_ENABLE true
                    UV_WRITE_MASK_DEPTH true
                    #UV_BLEND_FUNC GL_ONE GL_ONE_MINUS_SRC_ALPHA
                }
             }
        }
    }
    pass #co2
    {
        passEnable true

        useDataObject co2Data

        shader
        {
            type defaultMeshShader
            {        
                stateManagerVar __objectName__.Alpha                uv_alpha
                StateManagerVar __objectName__.simBindRealtime simBindRealtime
                StateManagerVar __objectName__.showTime showTime
                StateManagerVar __objectName__.co2Size co2Size
                StateManagerVar __objectName__.co2Color co2Color

                vertexShader ./modules/$folderName/co2_shader.vs
                fragmentShader ./modules/$folderName/co2_shader.fs                
                geometryShader ./modules/$folderName/co2_shader.gs

                glState
                {
                    UV_CULL_FACE_ENABLE true
                    UV_DEPTH_ENABLE false
                    UV_BLEND_ENABLE true
                    UV_WRITE_MASK_DEPTH true
                    #UV_BLEND_FUNC GL_ONE GL_ONE_MINUS_SRC_ALPHA
                }
             }
        }
    }

}

'''

## UVStream

In [397]:
UVSTREAM='''<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<DataStream version="0.1">
    <PrimitiveType>points</PrimitiveType>
    <DataBindings>
        <Variable>
            <Name>begin_year</Name>         
            <Type>float</Type>
        </Variable>             
        <Variable>
            <Name>end_year</Name>         
            <Type>float</Type>
        </Variable>             
        <Variable>
            <Name>longitude</Name>         
            <Type>float</Type>
        </Variable>             
        <Variable>
            <Name>latitude</Name>         
            <Type>float</Type>
        </Variable>             
        <Variable>
            <Name>rndFive</Name>         
            <Type>float</Type>
        </Variable>             
    </DataBindings>

    <CSVData>
        <Path>pop.csv</Path>
        <Headers>
            <Variable>              
                <Cols>0</Cols>
            </Variable>
            <Variable>              
                <Cols>1</Cols>
            </Variable>                     
            <Variable>              
                <Cols>2</Cols>
            </Variable>                     
            <Variable>              
                <Cols>3</Cols>
            </Variable>                     
            <Variable>              
                <Cols>4</Cols>
            </Variable>                     
         </Headers>
    </CSVData>
</DataStream>  
'''

In [398]:
uvStreamFile=open('data/pop.uvstream','w')
uvStreamFile.write(UVSTREAM)
uvStreamFile.close()

In [399]:
UVSTREAM='''<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<DataStream version="0.1">
    <PrimitiveType>points</PrimitiveType>
    <DataBindings>
        <Variable>
            <Name>year</Name>         
            <Type>float</Type>
        </Variable>                         
        <Variable>
            <Name>longitude</Name>         
            <Type>float</Type>
        </Variable>             
        <Variable>
            <Name>latitude</Name>         
            <Type>float</Type>
        </Variable>             
        <Variable>
            <Name>co2</Name>         
            <Type>float</Type>
        </Variable>             
    </DataBindings>

    <CSVData>
        <Path>co2.csv</Path>
        <Headers>
            <Variable>              
                <Cols>0</Cols>
            </Variable>
            <Variable>              
                <Cols>1</Cols>
            </Variable>                     
            <Variable>              
                <Cols>2</Cols>
            </Variable>                     
            <Variable>              
                <Cols>3</Cols>
            </Variable>                                        
         </Headers>
    </CSVData>
</DataStream>  
'''

In [400]:
uvStreamFile=open('data/co2.uvstream','w')
uvStreamFile.write(UVSTREAM)
uvStreamFile.close()

### Population and CO2 Class

In [401]:
class popClass():
    def __init__(self, object):
        self.object = object
        uvmod.Utility.ensurerelativepathexsists("pop_shader.gs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("pop_shader.vs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("pop_shader.fs",dataFolder)
        self.ColorMultiplier = "1 1 1 1"
        self.Scale = 1
        self.cr = 1000
    def setColorMultiplier(self,ColorMultiplier):
        self.ColorMultiplier = " ".join(str(c) for c in ColorMultiplier)
    def setScale(self,Scale):
        self.Scale = Scale
    def setRotationAngle(self,RotationAngle):
        self.RotationAngle = RotationAngle
    def generatemod(self):
        self.object.setgeometry(self.object.name+"Mesh.usesconf")
        return self.object.generatemod()
    def generatefiles(self, absOutDir, relOutDir):
        fileName = self.object.name+"Mesh.usesconf"
        s = string.Template(popTemplate)
        f = open(absOutDir+"\\"+fileName, 'w')
        if f:
            f.write(s.substitute(folderName = relOutDir,
                                 cr = self.cr,
                                 ColorMultiplier = self.ColorMultiplier,
                                 Scale = self.Scale))
            f.close()
        uvmod.Utility.copyfoldercontents(os.getcwd()+"\\"+dataFolder, absOutDir)

### Object Instantiation

In [402]:
popMap = popClass(uvmod.OrbitalObject())
generator = uvmod.Generator()
scene = uvmod.Scene()
modinfo = uvmod.ModuleInformation()

### Specify Settings and generate the module

In [403]:
popMap.object.showatstartup(True)
popMap.object.setcameraradius(10)
popMap.object.setcoord("Earth")
popMap.object.setname("PopAndCO2")
popMap.object.setguiname("/KavliLecture/Schrag/Population and CO2")
popMap.object.settargetradius(20)
popMap.object.showatstartup(False)
popMap.setColorMultiplier((1.,1.,1.,1.))
popMap.setScale(100)
popMap.cr = 10000
popMap.object.custom = "bin 9999"
popMap.object.custom += """
    rotation
	{
		spice
		{
			frame		IAU_EARTH 
			reference	ECLIPJ2000
		}
	}"""


modinfo.setname("Population and CO2")
modinfo.setauthor("Mark SubbaRao<sup>1</sup> and Daniel Schrag<sup>2</sup><br />(1)Adler Planetarium,<br />(2)Harvard University")
modinfo.cleardependencies()
modinfo.setdesc("Population and CO2 emmissions over time. Each dot represents 1 million people<br /> \
                The data are from here: http://worldpopulationhistory.org/ <br/> ")
#modinfo.setthumbnail("data/thumb.png")
modinfo.setversion("1.0")

generator.generate("PopAndCO2",[],[popMap],modinfo)
uvmod.Utility.senduvcommand(popMap.object.name+".reload")


### Helper Functions for modifing code
##### Reload Module and Shaders in Uniview

In [404]:
uvmod.Utility.senduvcommand(popMap.object.name+".reload")


Copy modified Shader files and reload

In [405]:
from config import Settings


In [406]:
uvmod.Utility.copyfoldercontents(os.getcwd()+"\\"+dataFolder, Settings.uvcustommodulelocation+'\\'+popMap.object.name)
uvmod.Utility.senduvcommand(popMap.object.name+".reload")